In [1]:
import psycopg2
import psycopg2.extras as extras
import config_warehouse as creds
import config_lake as creds_lake
import pandas as pd
import numpy as np

In [2]:
# Set up a connection to the warehouse
conn_string = "host="+ creds.PGHOST +" port="+ "5432" +" dbname="+ creds.PGDATABASE +" user=" + creds.PGUSER \
          +" password="+ creds.PGPASSWORD

conn = psycopg2.connect(conn_string)
print("Connected!")

# Create a cursor object
cursor = conn.cursor()

# Select table measuring points
cursor.execute((""" SELECT * FROM measuring_points """))
df_records = pd.DataFrame(cursor.fetchall())
df_measuring_points = pd.DataFrame(df_records)
colnames = [desc[0] for desc in cursor.description]
#print(colnames)
df_measuring_points.columns = colnames

# Select table measuring points location
cursor.execute((""" SELECT * FROM measuring_points_coordinates """))
df_records = pd.DataFrame(cursor.fetchall())
df_measuring_points_coordinates = pd.DataFrame(df_records)
colnames = [desc[0] for desc in cursor.description]
df_measuring_points_coordinates.columns = colnames

# Select table traffic measurement
cursor.execute((""" SELECT * FROM traffic_measurement_21 """))
df_records = pd.DataFrame(cursor.fetchall())
df_traffic_21 = pd.DataFrame(df_records)
colnames = [desc[0] for desc in cursor.description]
df_traffic_21.columns = colnames

# Select table charging_stations_static
cursor.execute((""" SELECT * FROM charging_stations_static """))
df_records = pd.DataFrame(cursor.fetchall())
df_charging_stations_static = pd.DataFrame(df_records)
colnames = [desc[0] for desc in cursor.description]
df_charging_stations_static.columns = colnames

# Select table charging_stations_location
cursor.execute((""" SELECT * FROM charging_stations_location """))
df_records = pd.DataFrame(cursor.fetchall())
df_charging_stations_location = pd.DataFrame(df_records)
colnames = [desc[0] for desc in cursor.description]
df_charging_stations_location.columns = colnames

conn.close()

Connected!


In [3]:
# # Overview
# print(df_measuring_points.head(),'\n',
# df_measuring_points_coordinates.head(), '\n',
# df_traffic_21.head(), '\n',
# df_charging_stations_static.head(), '\n',
# df_charging_stations_location.head())

In [4]:
#Merge dataframes # Measuring points w/ traffic
print('Lenght dataframes: ',
len(df_measuring_points),',',
len(df_measuring_points_coordinates),',',
len(df_traffic_21))
df1 = df_measuring_points.merge(df_measuring_points_coordinates, on = 'nr_measuring_point', how='left')
df_points = df_traffic_21.merge(df1, on = 'nr_measuring_point', how= 'left')
df_points.drop_duplicates(subset=['nr_measuring_point'], inplace=True)
df_points.drop(['nr_x', 'nr_y'], axis=1, inplace=True)
df_points

Lenght dataframes:  459 , 918 , 386


,nr_measuring_point,annual_average,point_name,status,type_point,road,canton_abbreviation,coordinate_east,coordinate_nord
0,2,0,CHALET-A-GOBET,in Betrieb,Online+Statistik,H 1,VD,2545269,1158463
2,5,6918,SCHWANDEN N,in Betrieb,Online+Statistik,H 17,GL,2724500,1206725
4,9,7865,"BRUENIG, LETZI",in Betrieb,Online+Statistik,A 8,OW,2653895,1180111
6,10,0,"HOSPENTAL S, ST. GOTTHARD",in Betrieb,Online+Statistik,H 2,UR,2686520,1163452
8,14,3434,"JULIER, LA VEDUTA",in Betrieb,Online+Statistik,H 3,GR,2780967,1148640
...,...,...,...,...,...,...,...,...,...
762,852,0,ECHANGEUR GD.ST.BERNARD EST (AR),in Betrieb,Online+Statistik,A 9,VS,2573706,1107161
764,853,0,BIENNE NORD (AB),in Betrieb,Online+Statistik,A 16,BE,2587263,1222983
766,854,27123,BENKEN N (AB),in Betrieb,nur Statistik,A 4,ZH,2690786,1278336
768,855,27060,BENKEN S (AB),in Betrieb,nur Statistik,A 4,ZH,2691429,1277447


In [5]:
df_points.isnull().values.any()

False

In [6]:
#Merge dataframes # Chargin stations w/ location
print('Lenght dataframes: ',
len(df_charging_stations_static),',',
len(df_charging_stations_location))
df_stations = df_charging_stations_static.merge(df_charging_stations_location, on='charging_station_id',
                                                how='left')
df_stations

Lenght dataframes:  5050 , 3826


,evse_id,charging_station_id,operator_id,city,street,power,id_postal_code,coordinate_east,coordinate_nord
0,CH*EWD*E123*0033,CH*EWD*E123*0033,CH*EWD,Davos,Bahnhofstrasse,22,1146,9.845910,46.813520
1,CH*EWD*E123*0027,CH*EWD*E123*0027,CH*EWD,Davos,Bahnhofstrasse,22,1146,9.845860,46.813500
2,CH*REPE116*01*1,CH*REPE116*01*1,CH*REP,Davos,Flüalastrasse,22,1146,9.851364,46.811108
3,CH*REPE116*01*2,CH*REPE116*01*1,CH*REP,Davos,Flüalastrasse,22,1146,9.851364,46.811108
4,CH*ECUEKWZFPH3QU3DLEHRXX5LBBLPNW2,CH*ECUEKWZFPH3QU3DLEHRXX5LBBLPNW2,CH*ECU,Davos Dorf,Hotel Inn Promenade 116,11,1146,9.836806,46.802939
...,...,...,...,...,...,...,...,...,...
5045,CH*ECUESQTDWS8E7GCXWWNLXPGKTY7XKW,CH*ECUESQTDWS8E7GCXWWNLXPGKTY7XKW,CH*ECU,Zürich,Reckenholzstrasse 191,22,1563,8.516089,47.429032
5046,CH*ECUESNV3BFTL6Q2L682S7HZFMSR6TQ,CH*ECUESNV3BFTL6Q2L682S7HZFMSR6TQ,CH*ECU,Domat/Ems,Via la Val 1,22,779,9.464860,46.837768
5047,CH*ECUELLVS9S9GYJTCYSNNLL8MBJSVYX,CH*ECUESNV3BFTL6Q2L682S7HZFMSR6TQ,CH*ECU,Domat/Ems,Via la Val 1,22,779,9.464860,46.837768
5048,CH*ECUETWY3BAC7LALSQ3YMWE9RPWQGHX,CH*ECUETWY3BAC7LALSQ3YMWE9RPWQGHX,CH*ECU,Luzern,Littauerboden 2,11,1508,8.264859,47.061965


In [7]:
df_stations.isnull().values.any()

False

In [8]:
# Approximate Formula to transform Swiss coordinates to WGS84 coordinates, deviation in ~1-3 meters
def lv95_to_wgs84_coordinates(y, x): # y = coordinate-east, x = coordinate-nord
    # convert into the civilian system in the unit 1000 km
    y = (y-2600000)/1000000
    x = (x-1200000)/1000000
    # Calculate longitude / latitude in the unit 10000"
    E = 2.6779094\
    + 4.728982 * y \
    + 0.791484 * y * x\
    + 0.1306 *y * x**2\
    - 0.0436 * y**3
    N = 16.9023892\
    + 3.238272 * x \
    - 0.270978 * y**2\
    - 0.002528 * x**2\
    - 0.0447 * y**2 * x\
    - 0.0140 * x**3
    #Convert longitude and latitude to unit °
    E = round(E * 100/36,6)
    N = round(N * 100/36,6)
    return [E, N]

# # Trial
# lv95_to_wgs84_coordinates(y = 2700000,
#                           x = 1100000)
# lv95_to_wgs84_coordinates(y = 2545269,
#                           x = 1158463)

In [9]:
# Transform coordinates
for row in range(0, len(df_points)):
    east = df_points.iloc[row, 7]
    nord = df_points.iloc[row, 8]
    #print(nord, east)
    coordinate_list = lv95_to_wgs84_coordinates(y = east, x = nord)
    #print(coordinate_list)
    df_points.loc[row, 'coordinate_east'] = coordinate_list[0]
    df_points.loc[row, 'coordinate_nord'] = coordinate_list[1]
df_points

,nr_measuring_point,annual_average,point_name,status,type_point,road,canton_abbreviation,coordinate_east,coordinate_nord
0,2.0,0.0,CHALET-A-GOBET,in Betrieb,Online+Statistik,H 1,VD,6.724671,46.575199
2,5.0,6918.0,SCHWANDEN N,in Betrieb,Online+Statistik,H 17,GL,8.144237,46.769994
4,9.0,7865.0,"BRUENIG, LETZI",in Betrieb,Online+Statistik,A 8,OW,9.794852,46.464632
6,10.0,0.0,"HOSPENTAL S, ST. GOTTHARD",in Betrieb,Online+Statistik,H 2,UR,8.101320,47.387874
8,14.0,3434.0,"JULIER, LA VEDUTA",in Betrieb,Online+Statistik,H 3,GR,8.541565,47.433075
...,...,...,...,...,...,...,...,...,...
377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.622008,47.691050
379,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.608638,47.691239
381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.098525,46.115434
383,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.646955,47.649383


In [ ]:
# Number of chargin stations within 5 km = 0.045 degrees
df_points['Charging_stations_5km'] = 0
df_points['Charging_stations_10km'] = 0
df_points['Charging_stations_15km'] = 0
df_points['Charging_stations_20km'] = 0
for row in range(0, len(df_points)):
    station_count_5km = 0
    station_count_10km = 0
    station_count_15km = 0
    station_count_20km = 0
    point_east = df_points.iloc[row, 7]
    point_nord = df_points.iloc[row, 8]
    for row in range(0, len(df_stations)):
        station_east = df_stations.iloc[row, 7]
        station_nord = df_stations.iloc[row, 8]
        if (point_east - station_east) <= 0.045 and \
                (point_nord - station_nord) <= 0.045:
            station_count_5km = station_count_5km + 1
        elif (point_east - station_east) <= 0.09 and \
                (point_nord - station_nord) <= 0.09:
            station_count_10km = station_count_10km + 1
        elif (point_east - station_east) <= 0.135 and \
                (point_nord - station_nord) <= 0.135:
            station_count_15km = station_count_15km + 1
        elif (point_east - station_east) <= 0.18 and \
                (point_nord - station_nord) <= 0.18:
            station_count_20km = station_count_20km + 1
        else:
            continue
    df_points.loc[row, 'Charging_stations_5km'] = station_count_5km
    df_points.loc[row, 'Charging_stations_10km'] = station_count_10km
    df_points.loc[row, 'Charging_stations_15km'] = station_count_15km
    df_points.loc[row, 'Charging_stations_20km'] = station_count_20km

df_points